In [1]:
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt

import seq2seq
from seq2seq import *

import re

datadir = 'data/'

/Users/jhara_mac/Desktop/plato/env/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def parse_file(filename='geobase.txt'):
    kb = {}
    with open(datadir+filename, 'r') as f:
        for line in f:
            if line.startswith('state'):
                m = re.match('state\((.*)\).\n', line.replace("'", ''))
                data = m.group(1).split(',')
                
                if 'state' not in kb:
                    kb['state'] = []
                
                relations = ['abbreviation', 'capital', 'population', 'area', \
                             'state_number', 'city', 'city', 'city', 'city']
                
                for rel, subj in zip(relations, data[1:]):
                    kb['state'].append((rel, subj, data[0]))

            elif line.startswith('city'):
                pass
            elif line.startswith('river'):
                pass
            elif line.startswith('border'):
                pass
            elif line.startswith('highlow'):
                pass
            elif line.startswith('mountain'):
                pass
            elif line.startswith('road'):
                pass
            elif line.startswith('lake'):
                pass
            else:
                pass
    return kb

In [3]:
kb = parse_file()

In [4]:
def generate_sentence(rel, subj, obj):
    return '%s is %s of %s' % (subj, rel, obj)

In [5]:
def generate_fol(rel, subj, obj):
    return '%s ( %s, %s )' % (rel, subj, obj)

In [6]:
src = []
tar = []
for tup in kb['state']:
    src.append(generate_sentence(*tup))
    tar.append(generate_fol(*tup))
print src[:10]
print tar[:10]

['al is abbreviation of alabama', 'montgomery is capital of alabama', '3894.0e+3 is population of alabama', '51.7e+3 is area of alabama', '22 is state_number of alabama', 'birmingham is city of alabama', 'mobile is city of alabama', 'montgomery is city of alabama', 'huntsville is city of alabama', 'ak is abbreviation of alaska']
['abbreviation ( al, alabama )', 'capital ( montgomery, alabama )', 'population ( 3894.0e+3, alabama )', 'area ( 51.7e+3, alabama )', 'state_number ( 22, alabama )', 'city ( birmingham, alabama )', 'city ( mobile, alabama )', 'city ( montgomery, alabama )', 'city ( huntsville, alabama )', 'abbreviation ( ak, alaska )']


In [8]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer()
tokenizer.fit_on_texts(src)
src_inputs = tokenizer.texts_to_sequences(src)
src_inputs = pad_sequences(src_inputs,
                           maxlen=max(len(seq) for seq in src_inputs))
src_m, src_n = src_inputs.shape
src_inputs = src_inputs.reshape((src_m, src_n, 1))

tokenizer = Tokenizer(filters='!"#$%&*+,-./:;<=>?@[\]^_`{|}~')
tokenizer.fit_on_texts(tar)
tar_inputs = tokenizer.texts_to_sequences(tar)
tar_inputs = pad_sequences(tar_inputs,
                           maxlen=max(len(seq) for seq in tar_inputs))
tar_m, tar_n = tar_inputs.shape
tar_inputs = tar_inputs.reshape((tar_m, tar_n, 1))

In [33]:
def test_seq2seq(src_inputs, tar_inputs, hidden_dim=100):
    _, input_length, input_dim = src_inputs.shape
    _, output_length, output_dim = tar_inputs.shape
    
    models = []
    
    # SIMPLE SEQ2SEQ
    
    # epoch 100: 7873.7063
    # models += [SimpleSeq2Seq(output_dim=output_dim, hidden_dim=hidden_dim, output_length=output_length, input_shape=(input_length, input_dim))]
    
    # models += [SimpleSeq2Seq(output_dim=output_dim, hidden_dim=hidden_dim, output_length=output_length, input_shape=(input_length, input_dim), depth=2)]
    # models += [SimpleSeq2Seq(output_dim=output_dim, hidden_dim=hidden_dim, output_length=output_length, input_shape=(input_length, input_dim), depth=4)]
    
    # epoch 100: 12396.6243
    # models += [SimpleSeq2Seq(output_dim=output_dim, hidden_dim=hidden_dim, output_length=output_length, input_shape=(input_length, input_dim), depth=8)]

    # SEQ2SEQ
 
    models += [Seq2Seq(output_dim=output_dim, hidden_dim=hidden_dim, output_length=output_length, input_shape=(input_length, input_dim))]
    models += [Seq2Seq(output_dim=output_dim, hidden_dim=hidden_dim, output_length=output_length, input_shape=(input_length, input_dim), peek=True)]
    models += [Seq2Seq(output_dim=output_dim, hidden_dim=hidden_dim, output_length=output_length, input_shape=(input_length, input_dim), depth=4)]
    # explodes
    models += [Seq2Seq(output_dim=output_dim, hidden_dim=hidden_dim, output_length=output_length, input_shape=(input_length, input_dim), peek=True, depth=4)]
    
    # Attention
    
    models += [AttentionSeq2Seq(output_dim=output_dim, hidden_dim=hidden_dim, output_length=output_length, input_shape=(input_length, input_dim))]
    #models += [AttentionSeq2Seq(output_dim=output_dim, hidden_dim=100, output_length=output_length, input_shape=(input_length, input_dim), depth=2)]
    #models += [AttentionSeq2Seq(output_dim=output_dim, hidden_dim=200, output_length=output_length, input_shape=(input_length, input_dim), depth=2)]
    #models += [AttentionSeq2Seq(output_dim=output_dim, hidden_dim=300, output_length=output_length, input_shape=(input_length, input_dim), depth=4)]
    
    histories = []
    for model in models:
        # model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['accuracy'])
        model.compile(loss='mse', optimizer='rmsprop', metrics=['accuracy'])
        history = model.fit(src_inputs, tar_inputs, nb_epoch=100)
        histories.append(history)
    return histories

In [34]:
histories = test_seq2seq(src_inputs, tar_inputs)

/Users/jhara_mac/Desktop/plato/env/lib/python2.7/site-packages/ipykernel_launcher.py:36: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Epoch 1/100
459/459 [==============================] - 17s 38ms/step - loss: 37.1995 - acc: 0.3387
Epoch 2/100
459/459 [==============================] - 0s 1ms/step - loss: 36.8491 - acc: 0.3527
Epoch 3/100
459/459 [==============================] - 0s 989us/step - loss: 36.7250 - acc: 0.3309
Epoch 4/100
459/459 [==============================] - 1s 1ms/step - loss: 36.7095 - acc: 0.3801
Epoch 5/100
459/459 [==============================] - 1s 1ms/step - loss: 36.6824 - acc: 0.4231
Epoch 6/100
459/459 [==============================] - 0s 1ms/step - loss: 36.6666 - acc: 0.4236
Epoch 7/100
459/459 [==============================] - 0s 1ms/step - loss: 36.6579 - acc: 0.4471
Epoch 8/100
459/459 [==============================] - 0s 1ms/step - loss: 36.6546 - acc: 0.4507
Epoch 9/100
459/459 [==============================] - 0s 1ms/step - loss: 36.6535 - acc: 0.4483
Epoch 10/100
459/459 [==============================] - 1s 1ms/step - loss: 36.6503 - acc: 0.4507
Epoch 11/100
459/459 [===

459/459 [==============================] - 1s 1ms/step - loss: 36.5926 - acc: 0.4602
Epoch 85/100
459/459 [==============================] - 1s 1ms/step - loss: 36.5911 - acc: 0.4602
Epoch 86/100
459/459 [==============================] - 1s 1ms/step - loss: 36.5935 - acc: 0.4602
Epoch 87/100
459/459 [==============================] - 0s 1ms/step - loss: 36.5925 - acc: 0.4602
Epoch 88/100
459/459 [==============================] - 1s 1ms/step - loss: 36.5936 - acc: 0.4602
Epoch 89/100
459/459 [==============================] - 0s 1ms/step - loss: 36.5946 - acc: 0.4602
Epoch 90/100
459/459 [==============================] - 0s 1ms/step - loss: 36.5923 - acc: 0.4602
Epoch 91/100
459/459 [==============================] - 0s 1ms/step - loss: 36.5933 - acc: 0.4602
Epoch 92/100
459/459 [==============================] - 0s 1ms/step - loss: 36.5922 - acc: 0.4602
Epoch 93/100
459/459 [==============================] - 0s 1ms/step - loss: 36.5914 - acc: 0.4602
Epoch 94/100
459/459 [===========

459/459 [==============================] - 0s 1ms/step - loss: 36.5917 - acc: 0.4602
Epoch 68/100
459/459 [==============================] - 0s 992us/step - loss: 36.5916 - acc: 0.4602
Epoch 69/100
459/459 [==============================] - 0s 1ms/step - loss: 36.5926 - acc: 0.4602
Epoch 70/100
459/459 [==============================] - 0s 997us/step - loss: 36.5914 - acc: 0.4602
Epoch 71/100
459/459 [==============================] - 0s 978us/step - loss: 36.5899 - acc: 0.4602
Epoch 72/100
459/459 [==============================] - 0s 1ms/step - loss: 36.5897 - acc: 0.4602
Epoch 73/100
459/459 [==============================] - 0s 995us/step - loss: 36.5898 - acc: 0.4602
Epoch 74/100
459/459 [==============================] - 0s 1ms/step - loss: 36.5907 - acc: 0.4602
Epoch 75/100
459/459 [==============================] - 1s 1ms/step - loss: 36.5907 - acc: 0.4602
Epoch 76/100
459/459 [==============================] - 0s 986us/step - loss: 36.5925 - acc: 0.4602
Epoch 77/100
459/459 [=

459/459 [==============================] - 1s 3ms/step - loss: 36.6858 - acc: 0.4038
Epoch 51/100
459/459 [==============================] - 1s 3ms/step - loss: 36.6894 - acc: 0.3955
Epoch 52/100
459/459 [==============================] - 2s 3ms/step - loss: 36.6831 - acc: 0.3997
Epoch 53/100
459/459 [==============================] - 2s 4ms/step - loss: 36.6811 - acc: 0.4079
Epoch 54/100
459/459 [==============================] - 2s 3ms/step - loss: 36.6864 - acc: 0.4096
Epoch 55/100
459/459 [==============================] - 2s 4ms/step - loss: 36.6846 - acc: 0.4018
Epoch 56/100
459/459 [==============================] - 2s 3ms/step - loss: 36.6841 - acc: 0.3958
Epoch 57/100
459/459 [==============================] - 2s 4ms/step - loss: 36.6878 - acc: 0.3951
Epoch 58/100
459/459 [==============================] - 1s 3ms/step - loss: 36.6811 - acc: 0.3948
Epoch 59/100
459/459 [==============================] - 1s 3ms/step - loss: 36.6811 - acc: 0.3985
Epoch 60/100
459/459 [===========

KeyboardInterrupt: 

In [ ]:
def plot_history(history):
    loss_list = [s for s in history.history.keys() if 'loss' in s and 'val' not in s]
    val_loss_list = [s for s in history.history.keys() if 'loss' in s and 'val' in s]
    acc_list = [s for s in history.history.keys() if 'acc' in s and 'val' not in s]
    val_acc_list = [s for s in history.history.keys() if 'acc' in s and 'val' in s]
    
    if len(loss_list) == 0:
        print('Loss is missing in history')
        return 
    
    ## As loss always exists
    epochs = range(1,len(history.history[loss_list[0]]) + 1)
    
    ## Loss
    plt.figure(1)
    for l in loss_list:
        plt.plot(epochs, history.history[l], 'b', label='Training loss (' + str(str(format(history.history[l][-1],'.5f'))+')'))
    for l in val_loss_list:
        plt.plot(epochs, history.history[l], 'g', label='Validation loss (' + str(str(format(history.history[l][-1],'.5f'))+')'))
    
    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    
    ## Accuracy
    plt.figure(2)
    for l in acc_list:
        plt.plot(epochs, history.history[l], 'b', label='Training accuracy (' + str(format(history.history[l][-1],'.5f'))+')')
    for l in val_acc_list:    
        plt.plot(epochs, history.history[l], 'g', label='Validation accuracy (' + str(format(history.history[l][-1],'.5f'))+')')

    plt.title('Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()

In [ ]:
# TESTING THE SIMPLE SEQ2SEQ
for history in histories:
    plot_history(history)

In [ ]:
# SEQ2SEQ
for history in histories:
    plot_history(history)